# This Notebook is Specifically for Constructing Unified Neuron-Level DataFrames Containing Measurements Stored In Multiple DataFrames


## Functions within the notebook:

#### def remove_unwanted_rows(NeuronIDlist, DF):
    This function selects only the rows associated with NeuronIDs that are present in the Neuron ID master list
    It requires 2 arguments:
        A) NeuronIDlist: The master list of neuron IDs represented in the entire dataset
        B) DF: A Neuron-Level Dataframe

#### def length_calibrated_DF(NEURONIDLIST, wantedcolumns, bDF):
    This function adds NaNs to the appropriate indices of the columns to be extracted (wantedcolumns) from the specific neuron-level dataframe (bDF) so that they can be merged into the MASTER NEURON-LEVEL DATAFRAME

#### def make_neuron_levelDF(listofDFs, NEURONIDLIST):
    This function adds the columns 




## Workflow
#### Merging data from sweep-level DataFrames:
    1. First, in a separate notebook (TIDY_DATAFRAMES, or Intrinsic_Properties) generate Neuron Level DataFrames

#### Merging data from pre-constructed Neuron-level DataFrames:
    1. Use the remove_unwanted_rows function to make sure that we aren't adding any NeuronIDs that are missing from the entirety of the rest of the dataset
    2. Use the length_calibrated_DF function to add NaN values where measurements, or NeuronIDs are missing from this specific DataFrame
    3. Use the make_neuron_levelDF function to add all of the desired columns from each unique DataFrame to the             neuron-level DataFrame

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

C:\Users\Ryan\Anaconda3\lib\site-packages\setuptools\_importlib.py:23: UserWarning: `importlib-metadata` version is incompatible with `setuptools`.
This problem is likely to be solved by installing an updated version of `importlib-metadata`.
  warnings.warn(msg)  # Ensure a descriptive message is shown.


In [2]:
#Read in the NeuronID Master List

NeuronIDlist = pd.read_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/NeuronLevel/FinalVersions/QualitativeData_Complete_07142022.csv')
NeuronIDlist = NeuronIDlist['NeuronID'].tolist()

In [3]:
#Generate ID_Region dictionary within notebook 

ID_region_DF = pd.read_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/NeuronLevel/FinalVersions/QualitativeData_Complete_07142022.csv')
ID_Region = dict(zip(ID_region_DF['NeuronID'],ID_region_DF['Updated Region']))
ID_Region

{'2020-09-22/RK/N2': 'FF',
 '2020-09-22/RK/N3': 'vZI',
 '2020-09-22/RK/N5': 'vZI',
 '2020-09-23/KM/N1': 'dZI',
 '2020-09-23/KM/N2': 'vZI',
 '2020-09-23/KM/N3': 'vLGN',
 '2020-09-23/RK/N1': 'vZI',
 '2020-09-23/RK/N2': 'vZI',
 '2020-09-23/RK/N3': 'vZI',
 '2020-09-23/RK/N4': 'vZI',
 '2020-09-23/RK/N5': 'vZI',
 '2020-09-23/RK/N6': 'vZI',
 '2020-09-24/KM/N1': 'd/vZI Border',
 '2020-09-24/KM/N2': 'd/vZI Border',
 '2020-09-24/KM/N3': 'vZI',
 '2020-09-24/KM/N4': 'vZI',
 '2020-09-24/KM/N5': 'vZI',
 '2020-09-24/KM/N6': 'd/vZI Border',
 '2020-10-01/KM/N1': 'dZI',
 '2020-10-01/KM/N2': 'dZI',
 '2020-10-01/KM/N3': 'vZI',
 '2020-10-01/KM/N4': 'vZI',
 '2020-10-01/KM/N5': 'dZI',
 '2020-10-01/KM/N6': 'dZI',
 '2020-10-01/RK/N1': 'dZI',
 '2020-10-01/RK/N2': 'vZI',
 '2020-10-01/RK/N3': 'dZI',
 '2020-10-01/RK/N4': 'dZI',
 '2020-10-01/RK/N5': 'cZI',
 '2020-10-01/RK/N6': 'cZI',
 '2020-10-02/KM/N1': 'dZI',
 '2020-10-02/KM/N2': 'd/vZI',
 '2020-10-02/KM/N3': 'vZI',
 '2020-10-02/KM/N4': 'STN',
 '2020-10-02/KM/N5'

In [75]:
## Neuron level DFs (containing NeuronID column)
ZI_RMP = pd.read_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/NeuronLevel/FinalVersions/2020-2022_RMP_07122022.csv').sort_values(by=['NeuronID', 'Filename'],axis=0)
ZI_Med_Cp = pd.read_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/NeuronLevel/FinalVersions/2020-2022_MedCp_07102022.csv').sort_values(by=['NeuronID'],axis=0)
ZI_FirstSpikeIF = pd.read_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/NeuronLevel/FinalVersions/2020-2022_First_IFSpike_DF_All.csv')#.sort_values(by=['NeuronID'],axis=0)
ZI_FirstSpikeGF = pd.read_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/NeuronLevel/FinalVersions/2020-2022_First_GapFreeSpike_DF_All.csv')#.sort_values(by=['NeuronID'],axis=0)
ZI_Ih = pd.read_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/NeuronLevel/FinalVersions/2020-2022_Ih_07102022.csv').sort_values(by=['NeuronID', 'Filename'],axis=0)
ZI_vsag = pd.read_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/NeuronLevel/FinalVersions/2020-2022_Vsag_07122022.csv').sort_values(by=['NeuronID', 'Filename'],axis=0)
ZI_RaRm = pd.read_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/NeuronLevel/FinalVersions/2020-2022_StartEnd_AveRaRm_08102022.csv').sort_values(by=['NeuronID', 'Filename'],axis=0)

In [76]:
DFlist = [ZI_RMP, ZI_Med_Cp, ZI_Ih, ZI_vsag, ZI_RaRm]

In [77]:
DFlist

[     Unnamed: 0 Cell Type        Date Mouse Neuron      Filename     Protocol  \
 4             4      Spp1  2020-09-22        RK/N2  20922002.abf  IC Gap Free   
 6             6      Spp1  2020-09-22        RK/N3  20922002.abf  IC Gap Free   
 5             5      Spp1  2020-09-22        RK/N3  20922004.abf  IC Gap Free   
 0             0      Spp1  2020-09-22        RK/N5  20922001.abf  IC Gap Free   
 1             1      Spp1  2020-09-22        RK/N5  20922002.abf  IC Gap Free   
 ..          ...       ...         ...   ...    ...           ...          ...   
 156         163   slc30a3  2022-06-17        KM/N3  22617019.abf  IC Gap Free   
 155         162   slc30a3  2022-06-17        KM/N4  22617028.abf  IC Gap Free   
 153         159   slc30a3  2022-06-17        KM/N5  22617037.abf  IC Gap Free   
 154         160   slc30a3  2022-06-17        RK/N2  _17_0005.abf  IC Gap Free   
 159         166   slc30a3  2022-06-17        15.ab  _17_0015.abf  IC Gap Free   
 
              

# ------------------------------------------------Functions------------------------------------------------

In [78]:
def remove_unwanted_rows(NeuronIDlist, DF):
    
    IDlist = [i for i in DF['NeuronID'] if i in NeuronIDlist]
    
    df = DF[DF['NeuronID'].isin(IDlist)].drop_duplicates('NeuronID')
    
    return df

In [79]:
def length_calibrated_DF(NeuronIDlist, wantedcolumns, specificDF):
    # Assign wantedcolumns (list) as the set of column headers from the specific dataframe (specificDF, e.g. IhDF, RaRmDF) 

    calibratedDF = pd.DataFrame()

    #K: checking what neurons ARE in the dictionary list but not the RaRm list
    missingIDs = [i for i in NeuronIDlist if i not in specificDF['NeuronID'].tolist()]
    
    
    #K: this will make a list of the indeces of the missing NEURONIDs
    idxlist = []

    for item in missingIDs:
        for idx, val in enumerate(NeuronIDlist):
            if val == item:
                idxlist.append(idx)      

    for item in wantedcolumns:
        if (item.find('Neuron') >= 0) & (item.find('ID') >= 0):
                
            y = specificDF.sort_values('NeuronID')[item].tolist()

            for n in idxlist:
                y.insert(n,NeuronIDlist[n])
            calibratedDF[item] = y
            
        else:
            x = specificDF[item].tolist()
            for n in idxlist:
                x.insert(n,'NaN')
            calibratedDF[item] = x
            
            
    return calibratedDF[wantedcolumns].drop_duplicates('NeuronID')

In [80]:
def make_neuron_levelDF(listofDFs, NeuronIDlist):
    
    neuron_levelDF = pd.DataFrame()
    neuron_levelDF['NeuronID'] = NeuronIDlist
    
    for DF in listofDFs:
        for item in DF.columns.tolist():
            if item.find('NeuronID') == -1:
                neuron_levelDF[item] = DF[item]
    
    return neuron_levelDF

# ---------------------------------Prepare DataFrames to Merge-----------------------------------

In [81]:
ZI_RaRm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117 entries, 0 to 116
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  117 non-null    int64  
 1   index                       117 non-null    int64  
 2   NeuronID                    117 non-null    object 
 3   Filename                    117 non-null    object 
 4   Cell Type                   117 non-null    object 
 5   Date                        117 non-null    object 
 6   Sweep Number                117 non-null    float64
 7   Start Holding Current (pA)  117 non-null    float64
 8   Start Average Ra (MOhms)    117 non-null    float64
 9   Start Average Rm (MOhms)    117 non-null    float64
 10  End Holding Current (pA)    117 non-null    float64
 11  End Average Ra (MOhms)      117 non-null    float64
 12  End Average Rm (MOhms)      117 non-null    float64
dtypes: float64(7), int64(2), object(4)


In [85]:
ZI_RaRm[ZI_RaRm['NeuronID' == '2020-09-23/RK/N1']]

KeyError: False

In [86]:
rkZI_RaRm = remove_unwanted_rows(NeuronIDlist, ZI_RaRm)

readyRm = length_calibrated_DF(NeuronIDlist, ['NeuronID',
                                              'Sweep Number',
                                              'Start Holding Current (pA)',
                                              'Start Average Ra (MOhms)',
                                              'Start Average Rm (MOhms)',
                                              'End Holding Current (pA)',
                                              'End Average Ra (MOhms)',
                                              'End Average Rm (MOhms)'], rkZI_RaRm)

len(readyRm)

117

In [87]:
rkZI_RMP = remove_unwanted_rows(NeuronIDlist, ZI_RMP)

readyRMP = length_calibrated_DF(NeuronIDlist, ['Date',  
                                               'Filepath', 
                                               'NeuronID', 
                                               'RMP 1', 
                                               'RMP 2'], rkZI_RMP)

len(readyRMP)

117

In [88]:
rkZI_Med_Cp = remove_unwanted_rows(NeuronIDlist, ZI_Med_Cp)

readyCP = length_calibrated_DF(NeuronIDlist, ['NeuronID', 
                                              'Capacitance (pF)'], rkZI_Med_Cp)

len(readyRMP)

117

In [89]:
ZI_FirstSpikeIF = ZI_FirstSpikeIF.rename(columns={'Neuron_ID':'NeuronID'})
ZI_FirstSpikeIF = ZI_FirstSpikeIF.rename(columns={'Half-Width (ms)':'IF_Half-width (ms)'})
ZI_FirstSpikeIF = ZI_FirstSpikeIF.rename(columns={'PeakAmplitude (mV)':'IF_PeakAmplitude (mV)'})
ZI_FirstSpikeIF = ZI_FirstSpikeIF.rename(columns={'Antipeak (mV)':'IF_Antipeak (mV)'})
ZI_FirstSpikeIF = ZI_FirstSpikeIF.rename(columns={'Max Rise Slope (upstroke)':'IF_Max Rise Slope (upstroke)'})
ZI_FirstSpikeIF = ZI_FirstSpikeIF.rename(columns={'Max Decay Slope (downstroke)':'IF_Max Decay Slope (downstroke)'})
ZI_FirstSpikeIF = ZI_FirstSpikeIF.rename(columns={'Upstroke_Downstroke Ratio':'IF_Upstroke_Downstroke Ratio'})

In [90]:
ZI_FirstSpikeIF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       121 non-null    int64  
 1   Date                             121 non-null    object 
 2   Neuron                           121 non-null    object 
 3   NeuronID                         121 non-null    object 
 4   Sweep                            121 non-null    int64  
 5   IF_Half-width (ms)               121 non-null    float64
 6   IF_PeakAmplitude (mV)            121 non-null    float64
 7   IF_Antipeak (mV)                 121 non-null    float64
 8   IF_Max Rise Slope (upstroke)     121 non-null    float64
 9   IF_Max Decay Slope (downstroke)  121 non-null    float64
 10  IF_Upstroke_Downstroke Ratio     121 non-null    float64
 11  Cell Type                        121 non-null    object 
dtypes: float64(6), int64(2

In [91]:
rkZI_FirstSpikeIF = remove_unwanted_rows(NeuronIDlist, ZI_FirstSpikeIF)

readyFirstSpikeIF = length_calibrated_DF(NeuronIDlist, ['NeuronID',
                                                        'IF_Half-width (ms)',
                                                        'IF_PeakAmplitude (mV)',
                                                        'IF_Antipeak (mV)',
                                                        'IF_Max Rise Slope (upstroke)',
                                                        'IF_Max Decay Slope (downstroke)',
                                                        'IF_Upstroke_Downstroke Ratio'], rkZI_FirstSpikeIF)

len(readyFirstSpikeIF)

117

In [92]:
ZI_FirstSpikeGF = ZI_FirstSpikeGF.rename(columns={'Neuron_ID':'NeuronID'})
ZI_FirstSpikeGF = ZI_FirstSpikeGF.rename(columns={'Half-Width (ms)':'GF_Half-width (ms)'})
ZI_FirstSpikeGF = ZI_FirstSpikeGF.rename(columns={'PeakAmplitude (mV)':'GF_PeakAmplitude (mV)'})
ZI_FirstSpikeGF = ZI_FirstSpikeGF.rename(columns={'Antipeak (mV)':'GF_Antipeak (mV)'})
ZI_FirstSpikeGF = ZI_FirstSpikeGF.rename(columns={'Max Rise Slope (upstroke)':'GF_Max Rise Slope (upstroke)'})
ZI_FirstSpikeGF = ZI_FirstSpikeGF.rename(columns={'Max Decay Slope (downstroke)':'GF_Max Decay Slope (downstroke)'})
ZI_FirstSpikeGF = ZI_FirstSpikeGF.rename(columns={'Upstroke_Downstroke Ratio':'GF_Upstroke_Downstroke Ratio'})

In [93]:
rkZI_FirstSpikeGF = remove_unwanted_rows(NeuronIDlist, ZI_FirstSpikeGF)

readyFirstSpikeGF = length_calibrated_DF(NeuronIDlist, ['NeuronID',
                                                        'GF_Half-width (ms)',
                                                        'GF_PeakAmplitude (mV)',
                                                        'GF_Antipeak (mV)',
                                                        'GF_Max Rise Slope (upstroke)',
                                                        'GF_Max Decay Slope (downstroke)',
                                                        'GF_Upstroke_Downstroke Ratio'], rkZI_FirstSpikeGF)

len(readyFirstSpikeGF)

117

In [94]:
rkZI_Ih = remove_unwanted_rows(NeuronIDlist, ZI_Ih)

readyIh = length_calibrated_DF(NeuronIDlist, ['NeuronID',
                                              'Ih_Ra (MOhms)',
                                              'Holding Current (pA)',
                                              'Ih (pA)',
                                              'Baseline Adjusted Tail Current (pA)'], rkZI_Ih)

len(readyIh)

117

In [95]:
rkZI_vsag = remove_unwanted_rows(NeuronIDlist, ZI_vsag)

readyVsag = length_calibrated_DF(NeuronIDlist,['NeuronID',
                                               'Holding Current (pA)',
                                               'Current Step (pA)',
                                               'Pre-step Membrane Voltage (mV)',
                                               'Peak Negative Membrane Voltage (mV)',
                                               'Voltage Sag (mV)'], rkZI_vsag)

len(readyVsag)

117

# ---------------------------------Merge The DataFrames-----------------------------------

In [96]:
QualitativeMeasures = pd.read_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/NeuronLevel/FinalVersions/QualitativeData_Complete_07142022.csv')

In [97]:
Neuron_Level_DF = make_neuron_levelDF([QualitativeMeasures, readyRMP, readyRm, readyCP, readyIh, readyVsag, readyFirstSpikeIF, readyFirstSpikeGF], NeuronIDlist)

In [98]:
Neuron_Level_DF.columns.to_list()

['NeuronID',
 'Unnamed: 0',
 'CellType',
 '4x DIC',
 'Found SDC?',
 'Updated Region',
 'Confidence of Region',
 'Exclusion Factor',
 'Section position from bregma (mm)',
 'Morphology',
 'Rebound Spike',
 'Spontaneous Activity',
 'Date',
 'Filepath',
 'RMP 1',
 'RMP 2',
 'Sweep Number',
 'Start Holding Current (pA)',
 'Start Average Ra (MOhms)',
 'Start Average Rm (MOhms)',
 'End Holding Current (pA)',
 'End Average Ra (MOhms)',
 'End Average Rm (MOhms)',
 'Capacitance (pF)',
 'Ih_Ra (MOhms)',
 'Holding Current (pA)',
 'Ih (pA)',
 'Baseline Adjusted Tail Current (pA)',
 'Current Step (pA)',
 'Pre-step Membrane Voltage (mV)',
 'Peak Negative Membrane Voltage (mV)',
 'Voltage Sag (mV)',
 'IF_Half-width (ms)',
 'IF_PeakAmplitude (mV)',
 'IF_Antipeak (mV)',
 'IF_Max Rise Slope (upstroke)',
 'IF_Max Decay Slope (downstroke)',
 'IF_Upstroke_Downstroke Ratio',
 'GF_Half-width (ms)',
 'GF_PeakAmplitude (mV)',
 'GF_Antipeak (mV)',
 'GF_Max Rise Slope (upstroke)',
 'GF_Max Decay Slope (downstroke

In [99]:
Neuron_Level_DF = Neuron_Level_DF.drop('Unnamed: 0', axis=1)

In [100]:
Neuron_Level_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 43 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   NeuronID                             117 non-null    object 
 1   CellType                             117 non-null    object 
 2   4x DIC                               117 non-null    object 
 3   Found SDC?                           117 non-null    object 
 4   Updated Region                       117 non-null    object 
 5   Confidence of Region                 117 non-null    object 
 6   Exclusion Factor                     9 non-null      object 
 7   Section position from bregma (mm)    98 non-null     float64
 8   Morphology                           52 non-null     object 
 9   Rebound Spike                        114 non-null    object 
 10  Spontaneous Activity                 116 non-null    object 
 11  Date                            

In [101]:
##KM: Only coerce the columns that do NOT require strings, only NaNs!!

cols =  ['RMP 1',
         'RMP 2',
         'Sweep Number',
         'Start Holding Current (pA)',
         'Start Average Ra (MOhms)',
         'Start Average Rm (MOhms)',
         'End Holding Current (pA)',
         'End Average Ra (MOhms)',
         'End Average Rm (MOhms)',
         'Capacitance (pF)',
         'Ih_Ra (MOhms)',
         'Holding Current (pA)',
         'Ih (pA)',
         'Baseline Adjusted Tail Current (pA)',
         'Current Step (pA)',
         'Pre-step Membrane Voltage (mV)',
         'Peak Negative Membrane Voltage (mV)',
         'Voltage Sag (mV)',
         'IF_Half-width (ms)',
         'IF_PeakAmplitude (mV)',
         'IF_Antipeak (mV)',
         'IF_Max Rise Slope (upstroke)',
         'IF_Max Decay Slope (downstroke)',
         'IF_Upstroke_Downstroke Ratio',
         'GF_Half-width (ms)',
         'GF_PeakAmplitude (mV)',
         'GF_Antipeak (mV)',
         'GF_Max Rise Slope (upstroke)',
         'GF_Max Decay Slope (downstroke)',
         'GF_Upstroke_Downstroke Ratio']

for item in cols:
     Neuron_Level_DF[item] = pd.to_numeric(Neuron_Level_DF[item], errors = 'coerce')

In [102]:
Neuron_Level_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 43 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   NeuronID                             117 non-null    object 
 1   CellType                             117 non-null    object 
 2   4x DIC                               117 non-null    object 
 3   Found SDC?                           117 non-null    object 
 4   Updated Region                       117 non-null    object 
 5   Confidence of Region                 117 non-null    object 
 6   Exclusion Factor                     9 non-null      object 
 7   Section position from bregma (mm)    98 non-null     float64
 8   Morphology                           52 non-null     object 
 9   Rebound Spike                        114 non-null    object 
 10  Spontaneous Activity                 116 non-null    object 
 11  Date                            

In [103]:
Neuron_Level_DF['Region_CellType'] = Neuron_Level_DF[['Updated Region', 'CellType']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [104]:
Neuron_Level_DF[Neuron_Level_DF['NeuronID'] == '2020-09-23/RK/N1']

,NeuronID,CellType,4x DIC,Found SDC?,Updated Region,Confidence of Region,Exclusion Factor,Section position from bregma (mm),Morphology,Rebound Spike,Spontaneous Activity,Date,Filepath,RMP 1,RMP 2,Sweep Number,Start Holding Current (pA),Start Average Ra (MOhms),Start Average Rm (MOhms),End Holding Current (pA),End Average Ra (MOhms),End Average Rm (MOhms),Capacitance (pF),Ih_Ra (MOhms),Holding Current (pA),Ih (pA),Baseline Adjusted Tail Current (pA),Current Step (pA),Pre-step Membrane Voltage (mV),Peak Negative Membrane Voltage (mV),Voltage Sag (mV),IF_Half-width (ms),IF_PeakAmplitude (mV),IF_Antipeak (mV),IF_Max Rise Slope (upstroke),IF_Max Decay Slope (downstroke),IF_Upstroke_Downstroke Ratio,GF_Half-width (ms),GF_PeakAmplitude (mV),GF_Antipeak (mV),GF_Max Rise Slope (upstroke),GF_Max Decay Slope (downstroke),GF_Upstroke_Downstroke Ratio,Region_CellType
6,2020-09-23/RK/N1,Spp1,Conclusive,NF,vZI,High,NaN,-2.8,NaN,No,Yes,2020-09-23,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,-50.653774,-66.870834,10.5,82.950563,11.657706,143.42937,82.950563,11.657706,143.42937,55.508992,10.084735,0.0,-41.87651,-156.28491,-100.0,-71.948326,-100.738525,1.975388,0.00076,18.76831,-60.852051,182.342541,-140.380871,1.298913,NaN,NaN,NaN,NaN,NaN,NaN,vZI Spp1


In [105]:
properendholding = []



for i in range(len(Neuron_Level_DF['End Holding Current (pA)'])):
    
    if Neuron_Level_DF['End Holding Current (pA)'].iloc[i] != Neuron_Level_DF['Start Holding Current (pA)'].iloc[i]:
        properendholding.append(Neuron_Level_DF['End Holding Current (pA)'].iloc[i])

    else: 
        properendholding.append('NaN')

Neuron_Level_DF['End Holding Current (pA)'] = properendholding

In [106]:
properendra = []


    
for i in range(len(Neuron_Level_DF['End Average Ra (MOhms)'])):
    
    if Neuron_Level_DF['End Average Ra (MOhms)'].iloc[i] != Neuron_Level_DF['Start Average Ra (MOhms)'].iloc[i]:
        properendra.append(Neuron_Level_DF['End Average Ra (MOhms)'].iloc[i])

    else: 
        properendra.append('NaN')

Neuron_Level_DF['End Average Ra (MOhms)'] = properendra

In [107]:
properendrm = []




for i in range(len(Neuron_Level_DF['End Average Rm (MOhms)'])):
    
    if Neuron_Level_DF['End Average Rm (MOhms)'].iloc[i] != Neuron_Level_DF['Start Average Rm (MOhms)'].iloc[i]:
        properendrm.append(Neuron_Level_DF['End Average Rm (MOhms)'].iloc[i])

    else: 
        properendrm.append('NaN')

Neuron_Level_DF['End Average Rm (MOhms)'] = properendrm       

In [108]:
Neuron_Level_DF.info()
Neuron_Level_DF

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 44 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   NeuronID                             117 non-null    object 
 1   CellType                             117 non-null    object 
 2   4x DIC                               117 non-null    object 
 3   Found SDC?                           117 non-null    object 
 4   Updated Region                       117 non-null    object 
 5   Confidence of Region                 117 non-null    object 
 6   Exclusion Factor                     9 non-null      object 
 7   Section position from bregma (mm)    98 non-null     float64
 8   Morphology                           52 non-null     object 
 9   Rebound Spike                        114 non-null    object 
 10  Spontaneous Activity                 116 non-null    object 
 11  Date                            

,NeuronID,CellType,4x DIC,Found SDC?,Updated Region,Confidence of Region,Exclusion Factor,Section position from bregma (mm),Morphology,Rebound Spike,Spontaneous Activity,Date,Filepath,RMP 1,RMP 2,Sweep Number,Start Holding Current (pA),Start Average Ra (MOhms),Start Average Rm (MOhms),End Holding Current (pA),End Average Ra (MOhms),End Average Rm (MOhms),Capacitance (pF),Ih_Ra (MOhms),Holding Current (pA),Ih (pA),Baseline Adjusted Tail Current (pA),Current Step (pA),Pre-step Membrane Voltage (mV),Peak Negative Membrane Voltage (mV),Voltage Sag (mV),IF_Half-width (ms),IF_PeakAmplitude (mV),IF_Antipeak (mV),IF_Max Rise Slope (upstroke),IF_Max Decay Slope (downstroke),IF_Upstroke_Downstroke Ratio,GF_Half-width (ms),GF_PeakAmplitude (mV),GF_Antipeak (mV),GF_Max Rise Slope (upstroke),GF_Max Decay Slope (downstroke),GF_Upstroke_Downstroke Ratio,Region_CellType
0,2020-09-22/RK/N2,Spp1,Conclusive,NF,FF,High,Wrong Region,NaN,NaN,Yes,Yes,2020-09-22,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,-49.206380,-57.254215,25.5,-61.024670,15.230345,228.634506,NaN,NaN,NaN,90.944294,16.989660,0.0,-113.940640,-35.452750,-150.0,-61.371710,-103.698730,12.408470,0.00072,43.579100,-61.431885,283.813500,-190.734875,1.488000,0.00080,29.687500,-65.844730,185.302738,-174.682625,1.060797,FF Spp1
1,2020-09-22/RK/N3,Spp1,Conclusive,NF,vZI,Medium,NaN,NaN,NaN,No,No,2020-09-22,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,-53.503020,-50.151070,2.0,-183.126430,12.868974,229.786248,-100.747681,12.188323,157.830345,95.445649,15.607772,0.0,-47.893066,-19.805523,-300.0,-54.247540,-100.921630,2.629417,0.00090,20.458984,-60.119629,166.503917,-112.670902,1.477790,NaN,NaN,NaN,NaN,NaN,NaN,vZI Spp1
2,2020-09-22/RK/N5,Spp1,Ambiguous,NF,vZI,Low,NaN,NaN,NaN,Yes,Yes,2020-09-22,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,-47.637640,-52.783570,10.5,-104.032883,11.243232,238.130767,0.325439,16.245239,392.665594,41.674380,23.059733,0.0,-25.706177,-8.993031,-150.0,-53.029050,-108.032230,2.926003,0.00075,23.730469,-61.364746,208.618175,-144.775398,1.440978,0.00100,16.619873,-56.506348,124.145513,-99.609382,1.246323,vZI Spp1
3,2020-09-23/KM/N1,Spp1,Conclusive,NF,dZI,Medium,NaN,NaN,NaN,No,No,2020-09-23,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,-54.831070,-58.017796,10.5,-227.711209,5.599573,96.675115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00084,22.583008,-61.126709,174.713146,-128.936779,1.355030,NaN,NaN,NaN,NaN,NaN,NaN,dZI Spp1
4,2020-09-23/KM/N2,Spp1,Conclusive,F,vZI,High,NaN,-2.70,Low,No,No,2020-09-23,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,-65.611220,-69.352430,10.5,-153.350571,8.913073,233.206144,-157.685773,12.554315,93.953947,NaN,14.170464,0.0,-72.742890,-26.285892,-150.0,-65.870930,-107.635500,8.252998,0.00232,32.257080,-59.753418,83.160406,-38.909915,2.137255,NaN,NaN,NaN,NaN,NaN,NaN,vZI Spp1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,2022-06-17/KM/N2,Slc30a3,Ambiguous,NF,d/vZI Border,High,NaN,-1.70,NaN,Yes,No,2022-06-17,/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,-40.228800,-51.100304,10.5,-20.978098,17.996699,1170.193412,-0.683002,19.74438,1175.06354,20.044732,19.305768,0.0,-7.421509,-8.384951,-45.0,-58.603477,-104.034424,0.645027,0.00140,28.839111,-62.103271,125.122080,-75.683598,1.653226,NaN,NaN,NaN,NaN,NaN,NaN,d/vZI Border Slc30a3
113,2022-06-17/KM/N3,Slc30a3,Conclusive,NF,dZI,High,NaN,-1.58,NaN,No,No,2022-06-17,/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,-49.645150,-61.097755,10.5,-46.967385,10.368155,431.474213,116.846428,23.915203,103.413292,32.276580,14.482281,0.0,-17.412140,-9.657829,-245.0,-65.080490,-104.949950,0.220161,0.00100,36.834717,-59.997559,179.443359,-117.187509,1.531250,NaN,NaN,NaN,NaN,NaN,NaN,dZI Slc30a3
114,2022-06-17/KM/N4,Slc30a3,Conclusive,F,dZI,High,NaN,-1.58,High,Yes,Yes,2022-06-17,/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,-45.270477,-53.692090,10

In [346]:
Neuron_Level_DF

,NeuronID,CellType,4x DIC,Found SDC?,Updated Region,Confidence of Region,Exclusion Factor,Section position from bregma (mm),Morphology,Rebound Spike,Spontaneous Activity,Date,Filepath,RMP 1,RMP 2,Sweep Number,Start Holding Current (pA),Start Average Ra (MOhms),Start Average Rm (MOhms),End Holding Current (pA),End Average Ra (MOhms),End Average Rm (MOhms),Capacitance (pF),Ih_Ra (MOhms),Holding Current (pA),Ih (pA),Baseline Adjusted Tail Current (pA),Current Step (pA),Pre-step Membrane Voltage (mV),Peak Negative Membrane Voltage (mV),Voltage Sag (mV),IF_Half-width (ms),IF_PeakAmplitude (mV),IF_Antipeak (mV),IF_Max Rise Slope (upstroke),IF_Max Decay Slope (downstroke),IF_Upstroke_Downstroke Ratio,GF_Half-width (ms),GF_PeakAmplitude (mV),GF_Antipeak (mV),GF_Max Rise Slope (upstroke),GF_Max Decay Slope (downstroke),GF_Upstroke_Downstroke Ratio,Region_CellType
0,2020-09-22/RK/N2,Spp1,Conclusive,NF,FF,High,Wrong Region,NaN,NaN,Yes,Yes,2020-09-22,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,-49.206380,-57.254215,25.5,-61.024670,15.230345,228.634506,0.325439,16.245239,392.665594,90.944294,16.989660,0.0,-113.940640,-35.452750,-150.0,-61.371710,-103.698730,12.408470,0.00072,43.579100,-61.431885,283.813500,-190.734875,1.488000,0.00080,29.687500,-65.844730,185.302738,-174.682625,1.060797,FF Spp1
1,2020-09-22/RK/N3,Spp1,Conclusive,NF,vZI,Medium,NaN,NaN,NaN,No,No,2020-09-22,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,-53.503020,-50.151070,2.0,-183.126430,12.868974,229.786248,-61.02467,15.230345,228.634506,95.445649,15.607772,0.0,-47.893066,-19.805523,-300.0,-54.247540,-100.921630,2.629417,0.00090,20.458984,-60.119629,166.503917,-112.670902,1.477790,NaN,NaN,NaN,NaN,NaN,NaN,vZI Spp1
2,2020-09-22/RK/N5,Spp1,Ambiguous,NF,vZI,Low,NaN,NaN,NaN,Yes,Yes,2020-09-22,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,-47.637640,-52.783570,10.5,-104.032883,11.243232,238.130767,-100.747681,12.188323,157.830345,41.674380,23.059733,0.0,-25.706177,-8.993031,-150.0,-53.029050,-108.032230,2.926003,0.00075,23.730469,-61.364746,208.618175,-144.775398,1.440978,0.00100,16.619873,-56.506348,124.145513,-99.609382,1.246323,vZI Spp1
3,2020-09-23/KM/N1,Spp1,Conclusive,NF,dZI,Medium,NaN,NaN,NaN,No,No,2020-09-23,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,-54.831070,-58.017796,10.5,-227.711209,5.599573,96.675115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00084,22.583008,-61.126709,174.713146,-128.936779,1.355030,NaN,NaN,NaN,NaN,NaN,NaN,dZI Spp1
4,2020-09-23/KM/N2,Spp1,Conclusive,F,vZI,High,NaN,-2.70,Low,No,No,2020-09-23,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,-65.611220,-69.352430,10.5,-153.350571,8.913073,233.206144,-9.136582,11.59113,193.367185,NaN,14.170464,0.0,-72.742890,-26.285892,-150.0,-65.870930,-107.635500,8.252998,0.00232,32.257080,-59.753418,83.160406,-38.909915,2.137255,NaN,NaN,NaN,NaN,NaN,NaN,vZI Spp1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,2022-06-17/KM/N2,Slc30a3,Ambiguous,NF,d/vZI Border,High,NaN,-1.70,NaN,Yes,No,2022-06-17,/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,-40.228800,-51.100304,10.5,-20.978098,17.996699,1170.193412,-0.683002,19.74438,1175.06354,20.044732,19.305768,0.0,-7.421509,-8.384951,-45.0,-58.603477,-104.034424,0.645027,0.00140,28.839111,-62.103271,125.122080,-75.683598,1.653226,NaN,NaN,NaN,NaN,NaN,NaN,d/vZI Border Slc30a3
113,2022-06-17/KM/N3,Slc30a3,Conclusive,NF,dZI,High,NaN,-1.58,NaN,No,No,2022-06-17,/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,-49.645150,-61.097755,10.5,-46.967385,10.368155,431.474213,116.846428,23.915203,103.413292,32.276580,14.482281,0.0,-17.412140,-9.657829,-245.0,-65.080490,-104.949950,0.220161,0.00100,36.834717,-59.997559,179.443359,-117.187509,1.531250,NaN,NaN,NaN,NaN,NaN,NaN,dZI Slc30a3
114,2022-06-17/KM/N4,Slc30a3,Conclusive,F,dZI,High,NaN,-1.58,High,Yes,Yes,2022-06-17,/Volumes/ryanKast/RKDATA/PatchData/ZIpatch/Slc...,-45.270

In [111]:
Neuron_Level_DF.to_csv('Z:/RKDATA/PatchData/ZIpatch/CompiledAnalysis/csv/NeuronLevel/FinalVersions/2020-2022_Merged_Neuron_Level_DF_08112022.csv')





## Spot Checking Neuron Measures





In [30]:
import random

pd.set_option('display.max_columns', None)

In [66]:
print(random.choice(NeuronIDlist))

2020-09-24/KM/N5


In [110]:


Neuron_Level_DF[Neuron_Level_DF['NeuronID'] == '2020-09-23/RK/N1']

,NeuronID,CellType,4x DIC,Found SDC?,Updated Region,Confidence of Region,Exclusion Factor,Section position from bregma (mm),Morphology,Rebound Spike,Spontaneous Activity,Date,Filepath,RMP 1,RMP 2,Sweep Number,Start Holding Current (pA),Start Average Ra (MOhms),Start Average Rm (MOhms),End Holding Current (pA),End Average Ra (MOhms),End Average Rm (MOhms),Capacitance (pF),Ih_Ra (MOhms),Holding Current (pA),Ih (pA),Baseline Adjusted Tail Current (pA),Current Step (pA),Pre-step Membrane Voltage (mV),Peak Negative Membrane Voltage (mV),Voltage Sag (mV),IF_Half-width (ms),IF_PeakAmplitude (mV),IF_Antipeak (mV),IF_Max Rise Slope (upstroke),IF_Max Decay Slope (downstroke),IF_Upstroke_Downstroke Ratio,GF_Half-width (ms),GF_PeakAmplitude (mV),GF_Antipeak (mV),GF_Max Rise Slope (upstroke),GF_Max Decay Slope (downstroke),GF_Upstroke_Downstroke Ratio,Region_CellType
6,2020-09-23/RK/N1,Spp1,Conclusive,NF,vZI,High,NaN,-2.8,NaN,No,Yes,2020-09-23,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,-50.653774,-66.870834,10.5,82.950563,11.657706,143.42937,NaN,NaN,NaN,55.508992,10.084735,0.0,-41.87651,-156.28491,-100.0,-71.948326,-100.738525,1.975388,0.00076,18.76831,-60.852051,182.342541,-140.380871,1.298913,NaN,NaN,NaN,NaN,NaN,NaN,vZI Spp1


In [80]:
QualitativeMeasures[QualitativeMeasures['NeuronID'] == '2020-09-24/KM/N5']

,Unnamed: 0,NeuronID,CellType,4x DIC,Found SDC?,Updated Region,Confidence of Region,Exclusion Factor,Section position from bregma (mm),Morphology,Rebound Spike,Spontaneous Activity
16,16,2020-09-24/KM/N5,Spp1,Conclusive,NF,vZI,High,NaN,-2.54,NaN,No,No


In [81]:
readyRMP[readyRMP['NeuronID'] == '2020-09-24/KM/N5']

,Date,Filepath,NeuronID,RMP 1,RMP 2
16,2020-09-24,/Volumes/ryanKast/RKDATA/PatchData/ZI/Spp1-Cre...,2020-09-24/KM/N5,-54.32051,-54.82769


In [83]:
readyRm[readyRm['NeuronID'] == '2020-09-24/KM/N5']

,NeuronID,Sweep Number,Holding Current (pA),Start Average Ra (MOhms),Start Average Rm (MOhms),End Average Ra (MOhms),End Average Rm (MOhms)
16,2020-09-24/KM/N5,10.5,-201.284467,2.960931,170.0114,2.960931,170.0114


In [82]:
readyCP[readyCP['NeuronID'] == '2020-09-24/KM/N5']

,NeuronID,Capacitance (pF)
16,2020-09-24/KM/N5,NaN


In [ ]:
re